In [12]:
#import modules
import scripts.config
from scripts.data_fetch_utils import fetch_sentiment_data

#Libraries
import pandas as pd
import numpy as np
import os
import datetime
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.gaussian_process.kernels import DotProduct, ConstantKernel as C, RationalQuadratic, ExpSineSquared, RBF, Sum, Product, WhiteKernel, ConstantKernel, Matern
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.metrics import precision_score, explained_variance_score, accuracy_score, classification_report, roc_auc_score, accuracy_score, log_loss, mean_squared_error, mean_absolute_error, r2_score, mean_squared_log_error
from sklearn.neural_network import MLPRegressor, MLPClassifier
from panelsplit import PanelSplit
from sklearn.impute import KNNImputer
import matplotlib.pyplot as plt
import seaborn as sns

# statsmodels for statistical methods
import statsmodels.tsa.api as smt
from statsmodels.stats.diagnostic import het_arch
import statsmodels.tsa.stattools as tsa
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA

#we use yfinance to fetch more stock data for sp500
import yfinance as yf
from yahoo_fin import stock_info as si

import requests
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

# # scipy stats for shapiro test
# from scipy import stats
# from pmdarima.arima import auto_arima
# from arch import arch_model

#Paths
path_in_ = r'/Users/amberwalker/SP500-Prediction-NN-GP/data'

In [ ]:
# # Fetch S&P 500 historical data
# sp500 = yf.Ticker("^GSPC")  # '^GSPC' is the ticker symbol for the S&P 500 index
# data = sp500.history(period="max")  # Get all available data
# print(data.tail())


                                  Open         High          Low        Close  \
Date                                                                            
2024-11-14 00:00:00-05:00  5989.680176  5993.879883  5942.279785  5949.169922   
2024-11-15 00:00:00-05:00  5912.790039  5915.319824  5853.009766  5870.620117   
2024-11-18 00:00:00-05:00  5874.169922  5908.120117  5865.950195  5893.620117   
2024-11-19 00:00:00-05:00  5870.049805  5923.509766  5855.290039  5916.979980   
2024-11-20 00:00:00-05:00  5914.339844  5920.669922  5860.560059  5917.109863   

                               Volume  Dividends  Stock Splits  
Date                                                            
2024-11-14 00:00:00-05:00  4184570000        0.0           0.0  
2024-11-15 00:00:00-05:00  4590960000        0.0           0.0  
2024-11-18 00:00:00-05:00  3983860000        0.0           0.0  
2024-11-19 00:00:00-05:00  4036940000        0.0           0.0  
2024-11-20 00:00:00-05:00  3772620000     

In [3]:
# tickers = "AAPL,GOOG"
# time_from = (datetime.now() - timedelta(days=30)).strftime("%Y%m%dT%H%M")
# time_to = datetime.now().strftime("%Y%m%dT%H%M")  # Current time

# # Base URL for the API
# base_url = "https://www.alphavantage.co/query"

# # Parameters
# params = {
#     "function": "NEWS_SENTIMENT",
#     "tickers": "AAPL,GOOG",  # Replace with desired tickers
#     "time_from": time_from,
#     "time_to": time_to,
#     "apikey": scripts.config.API_KEY,
# }

# # Fetch data
# response = requests.get(base_url, params=params)
# if response.status_code == 200:
#     data = response.json()
    
#     # Inspect and process the data
#     if "feed" in data:
#         news_df = pd.DataFrame(data["feed"])
#         print(news_df.head())  # View the first few rows
        
#         # Optionally, save the data to a CSV
#         news_df.to_csv("sentiment_data_last_month.csv", index=False)
#     else:
#         print("No news sentiment data available for the specified period.")
# else:
#     print(f"Error: {response.status_code}, Message: {response.text}")



In [7]:
# Parameters for fetching the data
tickers = "AAPL,GOOG"
time_from = (datetime.now() - timedelta(days=30)).strftime("%Y%m%dT%H%M")
time_to = datetime.now().strftime("%Y%m%dT%H%M")  # Current time

# Base URL for the API
base_url = "https://www.alphavantage.co/query"

# Parameters for the API request
params = {
    "function": "NEWS_SENTIMENT",
    "tickers": tickers,  # Tickers to filter news for
    "time_from": time_from,
    "time_to": time_to,
    "apikey": scripts.config.API_KEY,
}

# Fetch data from the API
response = requests.get(base_url, params=params)

if response.status_code == 200:
    data = response.json()
    
    if "feed" in data:
        # Convert the feed to a DataFrame
        news_df = pd.DataFrame(data["feed"])
        
        # Ensure ticker_sentiment field is in list format
        news_df = news_df.explode("ticker_sentiment")  # Expand each list in `ticker_sentiment` into separate rows
        
        # Extract individual ticker-related fields
        news_df["ticker"] = news_df["ticker_sentiment"].apply(lambda x: x.get("ticker") if isinstance(x, dict) else None)
        news_df["ticker_sentiment_score"] = news_df["ticker_sentiment"].apply(lambda x: x.get("ticker_sentiment_score") if isinstance(x, dict) else None)
        news_df["ticker_relevance_score"] = news_df["ticker_sentiment"].apply(lambda x: x.get("relevance_score") if isinstance(x, dict) else None)
        
        # Drop the original `ticker_sentiment` column if not needed
        news_df = news_df.drop(columns=["ticker_sentiment"])
        
        # Save to a CSV file or view the DataFrame
        news_df.to_csv("sentiment_data_with_tickers.csv", index=False)
        print(news_df.head())  # Print the first few rows of the DataFrame
    else:
        print("No news sentiment data available for the specified period.")
else:
    print(f"Error: {response.status_code}, Message: {response.text}")

                                               title  \
0  Nvidia Aggressively Bought, Russia Fires First...   
0  Nvidia Aggressively Bought, Russia Fires First...   
0  Nvidia Aggressively Bought, Russia Fires First...   
0  Nvidia Aggressively Bought, Russia Fires First...   
0  Nvidia Aggressively Bought, Russia Fires First...   

                                                 url   time_published  \
0  https://www.benzinga.com/markets/cryptocurrenc...  20241121T162416   
0  https://www.benzinga.com/markets/cryptocurrenc...  20241121T162416   
0  https://www.benzinga.com/markets/cryptocurrenc...  20241121T162416   
0  https://www.benzinga.com/markets/cryptocurrenc...  20241121T162416   
0  https://www.benzinga.com/markets/cryptocurrenc...  20241121T162416   

              authors                                            summary  \
0  [The Arora Report]  To gain an edge, this is what you need to know...   
0  [The Arora Report]  To gain an edge, this is what you need to know...

In [5]:
display(news_df.head())

,title,url,time_published,authors,summary,banner_image,source,category_within_source,source_domain,topics,overall_sentiment_score,overall_sentiment_label,ticker,ticker_sentiment_score,ticker_relevance_score
0,"Nvidia Aggressively Bought, Russia Fires First...",https://www.benzinga.com/markets/cryptocurrenc...,20241121T162416,[The Arora Report],"To gain an edge, this is what you need to know...",https://thearorareport.com/wp-content/uploads/...,Benzinga,General,www.benzinga.com,"[{'topic': 'Economy - Monetary', 'relevance_sc...",0.255217,Somewhat-Bullish,MSFT,0.172582,0.087344
0,"Nvidia Aggressively Bought, Russia Fires First...",https://www.benzinga.com/markets/cryptocurrenc...,20241121T162416,[The Arora Report],"To gain an edge, this is what you need to know...",https://thearorareport.com/wp-content/uploads/...,Benzinga,General,www.benzinga.com,"[{'topic': 'Economy - Monetary', 'relevance_sc...",0.255217,Somewhat-Bullish,FFXDF,0.076884,0.087344
0,"Nvidia Aggressively Bought, Russia Fires First...",https://www.benzinga.com/markets/cryptocurrenc...,20241121T162416,[The Arora Report],"To gain an edge, this is what you need to know...",https://thearorareport.com/wp-content/uploads/...,Benzinga,General,www.benzinga.com,"[{'topic': 'Economy - Monetary', 'relevance_sc...",0.255217,Somewhat-Bullish,GOOG,-0.244486,0.087344
0,"Nvidia Aggressively Bought, Russia Fires First...",https://www.benzinga.com/markets/cryptocurrenc...,20241121T162416,[The Arora Report],"To gain an edge, this is what you need to know...",https://thearorareport.com/wp-content/uploads/...,Benzinga,General,www.benzinga.com,"[{'topic': 'Economy - Monetary', 'relevance_sc...",0.255217,Somewhat-Bullish,META,0.172582,0.087344
0,"Nvidia Aggressively Bought, Russia Fires First...",https://www.benzinga.com/markets/cryptocurrenc...,20241121T162416,[The Arora Report],"To gain an edge, this is what you need to know...",https://thearorareport.com/wp-content/uploads/...,Benzinga,General,www.benzinga.com,"[{'topic': 'Economy - Monetary', 'relevance_sc...",0.255217,Somewhat-Bullish,NVDA,0.329884,0.298957


In [8]:
#let's build the features tha we'll actually need
news_df['time_published'] = pd.to_datetime(news_df['time_published'])
news_df['date'] = news_df['time_published'].dt.date
news_df['date'].head()

0    2024-11-21
0    2024-11-21
0    2024-11-21
0    2024-11-21
0    2024-11-21
Name: date, dtype: object

In [16]:
news_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 319 entries, 0 to 49
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   title                    319 non-null    object        
 1   url                      319 non-null    object        
 2   time_published           319 non-null    datetime64[ns]
 3   authors                  319 non-null    object        
 4   summary                  319 non-null    object        
 5   banner_image             319 non-null    object        
 6   source                   319 non-null    object        
 7   category_within_source   319 non-null    object        
 8   source_domain            319 non-null    object        
 9   topics                   319 non-null    object        
 10  overall_sentiment_score  319 non-null    float64       
 11  overall_sentiment_label  319 non-null    object        
 12  ticker                   319 non-null    o

In [20]:
news_df.dtypes

title                              object
url                                object
time_published             datetime64[ns]
authors                            object
summary                            object
banner_image                       object
source                             object
category_within_source             object
source_domain                      object
topics                             object
overall_sentiment_score           float64
overall_sentiment_label            object
ticker                             object
ticker_sentiment_score             object
ticker_relevance_score             object
date                               object
dtype: object

In [9]:
news_df[['overall_sentiment_score', 'ticker_sentiment_score', 'ticker_relevance_score']] = news_df[['overall_sentiment_score', 'ticker_sentiment_score', 'ticker_relevance_score']].astype(float)

In [10]:
#aggregate data to daily level (currently it's hourly)
daily_sentiment = news_df.groupby(['ticker', 'date']).agg({
    'ticker_sentiment_score': ['mean', 'sum'],
    'ticker_relevance_score': ['mean', 'sum'],
    'overall_sentiment_score': ['mean', 'sum'],

}).reset_index() #convert back to normal df

daily_sentiment.columns = ['ticker', 'date', 'avg_sentiment', 'total_sentiment', 'avg_relevance_score', 'total_relevance_score', 'avg_overall_sentiment', 'total_overall_sentiment']

daily_sentiment.head()



,ticker,date,avg_sentiment,total_sentiment,avg_relevance_score,total_relevance_score,avg_overall_sentiment,total_overall_sentiment
0,AAPL,2024-11-06,0.061544,0.184633,0.089788,0.269364,0.314280,0.942839
1,AAPL,2024-11-07,0.175829,1.054971,0.173328,1.039967,0.253399,1.520392
2,AAPL,2024-11-08,0.096665,0.386659,0.089933,0.359732,0.237092,0.948367
3,AAPL,2024-11-09,0.126462,0.126462,0.599092,0.599092,0.082913,0.082913
4,AAPL,2024-11-10,0.123267,0.123267,0.712808,0.712808,0.050468,0.050468


In [23]:
daily_sentiment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183 entries, 0 to 182
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ticker                   183 non-null    object 
 1   date                     183 non-null    object 
 2   avg_sentiment            183 non-null    float64
 3   total_sentiment          183 non-null    float64
 4   avg_relevance_score      183 non-null    float64
 5   total_relevance_score    183 non-null    float64
 6   avg_overall_sentiment    183 non-null    float64
 7   total_overall_sentiment  183 non-null    float64
dtypes: float64(6), object(2)
memory usage: 11.6+ KB


In [24]:
daily_sentiment['ticker'].unique()

array(['AAPL', 'ABNB', 'ACN', 'AFRM', 'AMGN', 'AMZN', 'APG', 'APP', 'BAC',
       'BOX', 'BRK-A', 'COIN', 'CRYPTO:BTC', 'CRYPTO:DOGE', 'CRYPTO:ETH',
       'CRYPTO:NANO', 'CSCO', 'DAL', 'DBX', 'DDOG', 'DOMO', 'DSSMY',
       'FOREX:CNY', 'FOREX:EUR', 'FOREX:GBP', 'FOREX:IDR', 'FOREX:JPY',
       'FOREX:USD', 'FOSL', 'GOOG', 'GRMN', 'GS', 'HON', 'HPQ', 'IHRT',
       'INTC', 'IT', 'IVZ', 'JNJ', 'LLY', 'META', 'MS', 'MSFT', 'NFLX',
       'NTDOF', 'NVDA', 'NVO', 'NYT', 'OMRNF', 'OPRA', 'PG', 'QCOM',
       'ROKU', 'SEKEF', 'SIRI', 'SMCI', 'SNEJF', 'SONO', 'SPOT', 'SQ',
       'SSNLF', 'SSTK', 'T', 'TSLA', 'TSM', 'TTD', 'UAL', 'UBER', 'UNH',
       'WEN', 'WLDS', 'WMT', 'XIACY'], dtype=object)

In [12]:
daily_sentiment_filtered = daily_sentiment.loc[daily_sentiment['ticker'].isin(['AAPL', 'GOOG'])]

daily_sentiment_filtered.head()


,ticker,date,avg_sentiment,total_sentiment,avg_relevance_score,total_relevance_score,avg_overall_sentiment,total_overall_sentiment
0,AAPL,2024-11-06,0.061544,0.184633,0.089788,0.269364,0.314280,0.942839
1,AAPL,2024-11-07,0.175829,1.054971,0.173328,1.039967,0.253399,1.520392
2,AAPL,2024-11-08,0.096665,0.386659,0.089933,0.359732,0.237092,0.948367
3,AAPL,2024-11-09,0.126462,0.126462,0.599092,0.599092,0.082913,0.082913
4,AAPL,2024-11-10,0.123267,0.123267,0.712808,0.712808,0.050468,0.050468


In [13]:
daily_sentiment_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32 entries, 0 to 80
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ticker                   32 non-null     object 
 1   date                     32 non-null     object 
 2   avg_sentiment            32 non-null     float64
 3   total_sentiment          32 non-null     float64
 4   avg_relevance_score      32 non-null     float64
 5   total_relevance_score    32 non-null     float64
 6   avg_overall_sentiment    32 non-null     float64
 7   total_overall_sentiment  32 non-null     float64
dtypes: float64(6), object(2)
memory usage: 2.2+ KB


In [3]:
#let's extract the sp500 tickers from Wikipedia
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
sp500_companies = pd.read_html(url)[0]
tickers = sp500_companies['Symbol'].to_list()
print(tickers)

['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A', 'APD', 'ABNB', 'AKAM', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AMTM', 'AEE', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA', 'AAPL', 'AMAT', 'APTV', 'ACGL', 'ADM', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'AXON', 'BKR', 'BALL', 'BAC', 'BAX', 'BDX', 'BRK.B', 'BBY', 'TECH', 'BIIB', 'BLK', 'BX', 'BK', 'BA', 'BKNG', 'BWA', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF.B', 'BLDR', 'BG', 'BXP', 'CHRW', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'COR', 'CNC', 'CNP', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CPAY', 'CTVA', 'CSGP', 'COST', 'CTRA', 'CRWD', 'CCI', 'CSX', 'CMI', 'CVS', 'DHR', '

In [2]:
time_from = (datetime.now() - timedelta(days=90)).strftime("%Y%m%dT%H%M")
time_to = datetime.now().strftime("%Y%m%dT%H%M")

news_df = fetch_sentiment_data(time_from, time_to)
news_df.head()

,ticker,date,avg_sentiment,total_sentiment,avg_relevance_score,total_relevance_score,avg_overall_sentiment,total_overall_sentiment
0,AAPL,2024-11-06,0.061544,0.184633,0.089788,0.269364,0.314280,0.942839
1,AAPL,2024-11-07,0.175829,1.054971,0.173328,1.039967,0.253399,1.520392
2,AAPL,2024-11-08,0.096665,0.386659,0.089933,0.359732,0.237092,0.948367
3,AAPL,2024-11-09,0.126462,0.126462,0.599092,0.599092,0.082913,0.082913
4,AAPL,2024-11-10,0.123267,0.123267,0.712808,0.712808,0.050468,0.050468


In [5]:
news_df.sort_values(by=['date'], inplace=True)
news_df.head()

,ticker,date,avg_sentiment,total_sentiment,avg_relevance_score,total_relevance_score,avg_overall_sentiment,total_overall_sentiment
0,AAPL,2024-11-06,0.061544,0.184633,0.089788,0.269364,0.314280,0.942839
65,GOOG,2024-11-06,0.267805,0.803416,0.199555,0.598666,0.314280,0.942839
66,GOOG,2024-11-07,0.116494,0.698963,0.109966,0.659795,0.253399,1.520392
1,AAPL,2024-11-07,0.175829,1.054971,0.173328,1.039967,0.253399,1.520392
67,GOOG,2024-11-08,0.044559,0.178237,0.055613,0.222453,0.237092,0.948367


In [6]:
news_df.tail()

,ticker,date,avg_sentiment,total_sentiment,avg_relevance_score,total_relevance_score,avg_overall_sentiment,total_overall_sentiment
13,AAPL,2024-11-19,-0.007878,-0.031512,0.217551,0.870203,0.225752,0.903010
79,GOOG,2024-11-20,0.159183,0.636733,0.326850,1.307400,0.204043,0.816171
14,AAPL,2024-11-20,0.064307,0.257227,0.256194,1.024777,0.204043,0.816171
15,AAPL,2024-11-21,0.149479,0.448437,0.119660,0.358981,0.149465,0.448396
80,GOOG,2024-11-21,-0.088808,-0.266424,0.293782,0.881345,0.149465,0.448396


In [48]:
from datetime import datetime as dt

# Define end_time
end_time = dt(2024, 8, 25)

print(end_time) 

2024-08-25 00:00:00


In [49]:
# # Define the time range
# end_time = dt(2024-08-25)
start_time = end_time - timedelta(days=90)

# Initialize an empty DataFrame to collect results
all_data = pd.DataFrame()

# Fetch data in 15-day intervals
while start_time < end_time:
    time_from = start_time.strftime("%Y%m%dT%H%M")
    time_to = (start_time + timedelta(days=15)).strftime("%Y%m%dT%H%M")
    
    # Ensure we don’t overshoot the end_time
    if start_time + timedelta(days=15) > end_time:
        time_to = end_time.strftime("%Y%m%dT%H%M")
    
    # Replace this with your actual API call
    data = fetch_sentiment_data(time_from, time_to)  # Example function
    
    # Append the fetched data to all_data
    all_data = pd.concat([all_data, data])
    
    # Update the start_time
    start_time += timedelta(days=15)

# Now `all_data` contains the concatenated results


No news sentiment data available for the specified period.


UnboundLocalError: cannot access local variable 'news_df' where it is not associated with a value

In [8]:
all_data.head()

,ticker,date,avg_sentiment,total_sentiment,avg_relevance_score,total_relevance_score,avg_overall_sentiment,total_overall_sentiment
0,AAPL,2024-08-25,0.280392,0.280392,0.612821,0.612821,0.223126,0.223126
1,AAPL,2024-08-26,0.089847,0.449233,0.245966,1.229828,0.171723,0.858613
2,AAPL,2024-08-27,0.019865,0.099326,0.134433,0.672164,0.234243,1.171217
3,AAPL,2024-08-28,0.083524,0.668189,0.104946,0.839571,0.217827,1.742616
4,AAPL,2024-08-29,0.075632,0.226895,0.071295,0.213884,0.180389,0.541167


In [44]:
all_data = pd.read_csv("data/3months_data.csv")

In [45]:
all_data.sort_values(['date'], inplace=True)
print(all_data.head())
print(all_data.tail())

   ticker        date  avg_sentiment  total_sentiment  avg_relevance_score  \
0    AAPL  2024-08-25       0.280392         0.280392             0.612821   
14   GOOG  2024-08-25       0.282880         0.282880             0.053035   
1    AAPL  2024-08-26       0.089847         0.449233             0.245966   
15   GOOG  2024-08-26       0.031979         0.159894             0.140194   
2    AAPL  2024-08-27       0.019865         0.099326             0.134433   

    total_relevance_score  avg_overall_sentiment  total_overall_sentiment  
0                0.612821               0.223126                 0.223126  
14               0.053035               0.223126                 0.223126  
1                1.229828               0.171723                 0.858613  
15               0.700971               0.171723                 0.858613  
2                0.672164               0.234243                 1.171217  
    ticker        date  avg_sentiment  total_sentiment  avg_relevance_score

In [11]:
all_data.to_csv('data/3months_data.csv', index=False)

In [9]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 158 entries, 0 to 78
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ticker                   158 non-null    object 
 1   date                     158 non-null    object 
 2   avg_sentiment            158 non-null    float64
 3   total_sentiment          158 non-null    float64
 4   avg_relevance_score      158 non-null    float64
 5   total_relevance_score    158 non-null    float64
 6   avg_overall_sentiment    158 non-null    float64
 7   total_overall_sentiment  158 non-null    float64
dtypes: float64(6), object(2)
memory usage: 11.1+ KB


In [ ]:
#first we build a df with all dates (including weekends) for EACH market
start_date = datetime.datetime(1999, 1, 1)
end_date = datetime.datetime(2020, 4, 30)
#date range
dates = pd.date_range(start=start_date, end=end_date, freq='D')
#all of the diff markets
markets = df_market['market'].unique()
#df build
date_df = pd.DataFrame([(date, market) for market in markets for date in dates], columns=['date', 'market'])

print(date_df.head())

In [18]:
import yfinance as yf
import pandas as pd

def calculate_book_to_market(ticker, start_date, end_date):
    """
    Calculates the daily Book-to-Market ratio for a given stock.
    
    Parameters:
        ticker (str): Stock ticker symbol (e.g., 'AAPL', 'GOOG').
        start_date (str): Start date for historical data (format: 'YYYY-MM-DD').
        end_date (str): End date for historical data (format: 'YYYY-MM-DD').
    
    Returns:
        pd.DataFrame: DataFrame with daily Book-to-Market ratio.
    """
    # Fetch daily stock data (price and shares outstanding)
    stock = yf.Ticker(ticker)
    historical = stock.history(start=start_date, end=end_date)
    
    if historical.empty:
        raise ValueError(f"No data found for ticker {ticker} between {start_date} and {end_date}")
    
    # Calculate daily market cap: Close Price × Shares Outstanding
    historical['market_cap'] = historical['Close'] * stock.info['sharesOutstanding']
    
    # Fetch quarterly book value data
    quarterly_financials = stock.quarterly_balance_sheet
    if 'Total Stockholder Equity' not in quarterly_financials.index:
        raise ValueError(f"Book value (Stockholder Equity) not available for {ticker}")
    
    # Extract book value and align with dates
    book_value = quarterly_financials.loc['Total Stockholder Equity']
    book_value = book_value.transpose().reset_index()
    book_value.columns = ['quarter_end', 'book_value']
    book_value['quarter_end'] = pd.to_datetime(book_value['quarter_end'])
    
    # Forward-fill book value for each date
    historical['date'] = historical.index
    historical['book_value'] = historical['date'].apply(
        lambda x: book_value.loc[book_value['quarter_end'] <= x, 'book_value'].max()
    )
    
    # Calculate daily Book-to-Market ratio
    historical['book_to_market_ratio'] = historical['book_value'] / historical['market_cap']
    
    # Return the DataFrame with relevant columns
    return historical[['date', 'market_cap', 'book_value', 'book_to_market_ratio']].dropna()


In [20]:
# Parameters
ticker = 'GOOG'  # Example: Apple
start_date = '2022-01-01'
end_date = '2023-12-31'

# Calculate B/M Ratio
bm_data = calculate_book_to_market(ticker, start_date, end_date)
print(bm_data.head())


ValueError: Book value (Stockholder Equity) not available for GOOG

In [24]:
import yahoo_fin.stock_info as si
apple = yf.Ticker('GOOG')
dict =  apple.info
df = pd.DataFrame.from_dict(dict,orient='index')
df = df.reset_index()

In [25]:
df.head()

,index,0
0,address1,1600 Amphitheatre Parkway
1,city,Mountain View
2,state,CA
3,zip,94043
4,country,United States


In [27]:
from yahooquery import Ticker

symbol_list_yahoo = ['META', 'AAPL', 'AMZN', 'NFLX', 'GOOG']

ticker = Ticker(symbol_list_yahoo) 
fin_data_dict = ticker.financial_data
fin_data_df = pd.DataFrame.from_dict(fin_data_dict, orient=‘index’).T

SyntaxError: invalid character '‘' (U+2018) (3571416342.py, line 7)

In [28]:
fin_data_dict

NameError: name 'fin_data_dict' is not defined

In [35]:
sp500_sent_data = pd.read_csv("/Users/amberwalker/SP500-Prediction-NN-GP/data/sp500_daily_ratios_20yrs.csv")
sp500_sent_data.head()

,Ticker,Date,Open,Close,Volume,quarter,year,Asset Turnover,Current Ratio,Days Sales In Receivables,...,Gross Margin,Inventory Turnover Ratio,Long-term Debt / Capital,Net Profit Margin,Operating Margin,Pre-Tax Profit Margin,ROA - Return On Assets,ROE - Return On Equity,Receiveable Turnover,Return On Tangible Equity
0,A,2005-04-01,13.88,13.67,4717800,2,2005,0.1758,2.7269,67.8873,...,50.0783,0.666,0.2291,7.4335,3.2864,4.4601,0.7292,1.3699,1.3257,1.5538
1,A,2005-04-04,13.59,13.26,6827100,2,2005,0.1758,2.7269,67.8873,...,50.0783,0.666,0.2291,7.4335,3.2864,4.4601,0.7292,1.3699,1.3257,1.5538
2,A,2005-04-05,13.29,13.23,5839100,2,2005,0.1758,2.7269,67.8873,...,50.0783,0.666,0.2291,7.4335,3.2864,4.4601,0.7292,1.3699,1.3257,1.5538
3,A,2005-04-06,13.23,13.42,2994600,2,2005,0.1758,2.7269,67.8873,...,50.0783,0.666,0.2291,7.4335,3.2864,4.4601,0.7292,1.3699,1.3257,1.5538
4,A,2005-04-07,13.38,13.37,4072300,2,2005,0.1758,2.7269,67.8873,...,50.0783,0.666,0.2291,7.4335,3.2864,4.4601,0.7292,1.3699,1.3257,1.5538


In [36]:
sp500_sent_data.columns

Index(['Ticker', 'Date', 'Open', 'Close', 'Volume', 'quarter', 'year',
       'Asset Turnover', 'Current Ratio', 'Days Sales In Receivables',
       'Debt/Equity Ratio', 'EBIT Margin', 'EBITDA Margin', 'Gross Margin',
       'Inventory Turnover Ratio', 'Long-term Debt / Capital',
       'Net Profit Margin', 'Operating Margin', 'Pre-Tax Profit Margin',
       'ROA - Return On Assets', 'ROE - Return On Equity',
       'Receiveable Turnover', 'Return On Tangible Equity'],
      dtype='object')

In [37]:
sp500_news = pd.read_csv("/Users/amberwalker/SP500-Prediction-NN-GP/data/sp500_news_290k_articles.csv")
sp500_news.head()

,_id,compound,date,headline,neg,neu,pos,ticker,time
0,61ec1c1ad68efb091ce207ce,0.2960,Jan-09-22,What Type Of Shareholders Make Up Agilent Tech...,0.000,0.833,0.167,A,06:06AM
1,61ec1c1ad68efb091ce207ca,0.3612,Jan-21-22,"Asset Dedication, LLC Buys Invesco BulletShare...",0.000,0.828,0.172,A,09:38AM
2,61ec1c1bd68efb091ce207ec,0.0000,Nov-18-21,Agilent (A) to Report Q4 Earnings: What's in t...,0.000,1.000,0.000,A,09:02AM
3,61ec1c1bd68efb091ce207ef,0.0000,Nov-16-21,Alibaba (BABA) to Report Q2 Earnings: What's i...,0.000,1.000,0.000,A,10:23AM
4,61ec1c1dd68efb091ce2080d,-0.3182,Oct-13-21,Agilent Receives FDA Companion Diagnostic Appr...,0.205,0.651,0.144,A,04:16PM


In [38]:
sp500_news.sort_values(['date'], inplace=True)
sp500_news.head()

,_id,compound,date,headline,neg,neu,pos,ticker,time
22033,61ec1df5d68efb35643c83b7,0.0000,2019-12-19T00:00:00.000Z,Cordish Cos. leader confident in office projec...,0.000,1.000,0.000,L,09:13AM
21855,61ec1df5d68efb35643c83b6,0.0000,2020-01-02T00:00:00.000Z,"Despite Its High P/E Ratio, Is Loews Corporati...",0.000,1.000,0.000,L,12:39PM
22270,61ec1df5d68efb35643c83b5,0.6369,2020-01-22T00:00:00.000Z,Diamond Offshore to Release Fourth Quarter 201...,0.000,0.682,0.318,L,10:00AM
21637,61ec1df5d68efb35643c83b4,0.4404,2020-01-22T00:00:00.000Z,Loews Corporation to Release Fourth Quarter 20...,0.104,0.647,0.249,L,10:00AM
21566,61ec1df5d68efb35643c83b2,0.0000,2020-02-07T00:00:00.000Z,"Altium Packaging, LLC -- Moody's assigns Altiu...",0.000,1.000,0.000,L,12:17PM


In [39]:
sp500_news.tail()

,_id,compound,date,headline,neg,neu,pos,ticker,time
202626,637c05978a18397c87a8363f,0.4939,Sep-30-22,Fifth Third Bank Makes BLINK Festival Carbon N...,0.000,0.738,0.262,FITB,11:20AM
195645,637c03ff8a18397c87a7e76f,0.0000,Sep-30-22,"Futures Giant CME Considers Brokerage, Taking ...",0.000,1.000,0.000,CME,10:48PM
187935,6357197485163b3bdc075ce5,0.2023,Sep-30-22,1 Monster Opportunity in the Global Chip Shortage,0.204,0.510,0.286,AMAT,06:10AM
222051,637c07598a18397c87a88f18,0.0000,Sep-30-22,"US Earnings to Watch: Levi, Constellation Bran...",0.000,1.000,0.000,STZ,10:05AM
208796,637c06258a18397c87a85270,0.6369,Sep-30-22,10 Best Affordable Dividend Stocks to Buy,0.000,0.588,0.412,KEY,08:00AM
